In [20]:
import requests
import json
import random
import math
from datetime import datetime

DATABASE_URLS = {
    0: "https://lost-items-f914d-default-rtdb.firebaseio.com/items",
    1: "https://found-items-7dec3-default-rtdb.firebaseio.com/items",
    2: "https://resolved-items-default-rtdb.firebaseio.com/items"
    }

def randomChar(l):
    dicR = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
            "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
            "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    word = ""
    for i in range(l):
        word = word + random.choice(dicR)
    return word

def generate_id(status):
    # Determin prefix
    prefix = 'L' if status.lower() == 'lost' else 'F'
    ID_body = randomChar(10)
    itemID = f"{prefix}{ID_body}"
    return itemID

def calculate_square_bounds(lat, lon, side_meters=200):
    meters_per_degree_lat = 111000 
    delta_lat = (side_meters / 2) / meters_per_degree_lat
    delta_lon = (side_meters / 2) / (meters_per_degree_lat * math.cos(math.radians(lat)))
    min_lat = lat - delta_lat
    max_lat = lat + delta_lat
    min_lon = lon - delta_lon
    max_lon = lon + delta_lon
    
    return min_lat, max_lat, min_lon, max_lon

def go_through_DB(DB: str = "All"):
    DB = DB.lower()
    if DB == "lost":
        response = requests.get(f'{DATABASE_URLS[0]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "found":
        response = requests.get(f'{DATABASE_URLS[1]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "completed":
        response = requests.get(f'{DATABASE_URLS[2]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "all":
        for i in [0, 1, 2]:
            print(f"------------{i}------------")
            response = requests.get(f'{DATABASE_URLS[i]}.json')
            item_dict = response.json()
            for key, value in item_dict.items():
                print(f"Item ID: {key}")
                value["image"] = "Placeholder..."
                print(json.dumps(value, indent=2))
    else:
        print("Please set the DB to display. (All, Lost, Found, Completed)")

def search(filter:str, parameter:list):
    filter = filter.lower()
    if filter == "id" and len(parameter) == 1:
        item_id = parameter[0]
        if item_id.startswith("L"):
            index = 0
        else:
            index = 1
        search_dict = {"url": f"{DATABASE_URLS[index]}/{item_id}.json?", "id": item_id}
    elif filter == "type" and len(parameter) == 1:
        # item_type_options = [
        #     'Electronic Devices', 
        #     'Keys', 
        #     'Wallets and Identification Cards', 
        #     'Textbooks and Notebooks', 
        #     'Clothing', 
        #     'Eyeglasses and Sunglasses', 
        #     'Water Bottles and Lunch Boxes', 
        #     'Backpacks and Bags', 
        #     'Sporting Equipment', 
        #     'Umbrellas',
        #     'Other'
        # ]

        item_Cate = parameter[0]
        search_param = f'.json?orderBy="item_type"&equalTo="{item_Cate}"'
        search_list = []
        for key, url in DATABASE_URLS.items():
            search_list.append(f"{url}{search_param}")
    elif filter == "location" and len(parameter) == 2:
        search_dict = {}
        search_dict["bound"] = calculate_square_bounds(parameter[0], parameter[1])
        search_dict["urls"] = []
        search_param = f'.json?orderBy="latitude"&startAt={search_dict["bound"][0]}&endAt={search_dict["bound"][1]}'
        for key, url in DATABASE_URLS.items():
            search_dict["urls"].append(f"{url}{search_param}")
    elif filter == "date" and len(parameter) == 6:
        date_start = datetime(parameter[0], parameter[1],parameter[2])
        date_end = datetime(parameter[3], parameter[4],parameter[5])
        date_start_inclusive = datetime.combine(date_start, datetime.min.time()).timestamp()
        date_end_inclusive = datetime.combine(date_end, datetime.max.time()).timestamp()
        search_param = f'.json?orderBy="date"&startAt={date_start_inclusive}&endAt={date_end_inclusive}'
        search_list = []
        for key, url in DATABASE_URLS.items():
            search_list.append(f"{url}{search_param}")
    else:
        print("Invalid filter type or parameter!")
        return
    
    if filter == "id":
        response = requests.get(search_dict["url"])
        results = response.json()
        print(f"Item ID: {parameter[0]}")
        results["image"] = "Placeholder..."
        if value["date"]:
            date_from_stamp = datetime.fromtimestamp(value["date"]).strftime('%Y-%m-%d')
        else:
            date_from_stamp = "N/A"
        value["date"] = date_from_stamp
        print(json.dumps(results, indent=2))
    elif filter == "type" or filter == "date":
        results = {}
        for url in search_list:
            response = requests.get(url)
            results.update(response.json() or {})
        for key, value in results.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            if value["date"]:
                date_from_stamp = datetime.fromtimestamp(value["date"]).strftime('%Y-%m-%d')
            else:
                date_from_stamp = "N/A"
            value["date"] = date_from_stamp
            print(json.dumps(value, indent=2))
    elif filter == "location":
        results = {}
        for url in search_dict["urls"]:
            response = requests.get(url)
            results.update(response.json() or {})
        for key, value in results.items():
            if value['longitude'] >= search_dict["bound"][2] and\
                value['longitude'] <= search_dict["bound"][3]:
                print(f"Item ID: {key}")
                value["image"] = "Placeholder..."
                if value["date"]:
                    date_from_stamp = datetime.fromtimestamp(value["date"]).strftime('%Y-%m-%d')
                else:
                    date_from_stamp = "N/A"
                value["date"] = date_from_stamp
                print(json.dumps(value, indent=2))
    else:
        print("Invalid filter type!")
        return


def add_edit(item_json:dict, item_ID:str = None):
    if not item_ID:
        item_ID = generate_id(item_json["status"])
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        data = json.dumps(item_json)
        url = f"{url_base}/{item_ID}.json"
        response = requests.patch(url, data=data)
        if response.status_code == 200:
            print("Add successfull!!")
        else:
            print("Add unsuccessful!!")
    elif item_ID and len(item_ID) == 11 and item_ID[0] in ["L", "F"]:
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        item_json.pop("image")
        item_json.pop("latitude")
        item_json.pop("longitude")
        data = json.dumps(item_json)
        url = f"{url_base}/{item_ID}.json"
        response = requests.patch(url, data=data)
        if response.status_code == 200:
            print("Edit successfull!!")
        else:
            print("Edit unsuccessful!!")
    else:
        print("Wrong ID or JSON")
        
def delete(item_ID:str):
    for i in [0, 1, 2]:
        print(f"---------Database {i}------------")
        response = requests.delete(f'{DATABASE_URLS[i]}/{item_ID}.json')

def mark_as_resolved(item_ID:str):
    if item_ID and len(item_ID) == 11 and item_ID[0] in ["L", "F"]:
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        response = requests.get(f"{url_base}/{item_ID}.json")
        info = response.json()
        info["completed"] = True
        responseD = requests.delete(f"{url_base}/{item_ID}.json")
        responseA = requests.patch(f"{DATABASE_URLS[2]}/{item_ID}.json", data=json.dumps(info))


In [4]:
go_through_DB("lost")  # All, Lost, Found, Completed

Item ID: LXPoN6AmNdr
{
  "color": "Purple",
  "completed": false,
  "date": 1713250800.0,
  "description": "A Manager Add Test",
  "email": "Manager@example.com",
  "image": "Placeholder...",
  "item_type": "Scooter",
  "latitude": 0,
  "longitude": 0,
  "phone": "111111100000000",
  "status": "Lost"
}
Item ID: Lu7LqqhLMhU
{
  "color": "Black",
  "completed": false,
  "date": 1712788621.027968,
  "description": "A Scooter",
  "email": "scooter@example.com",
  "image": "Placeholder...",
  "item_type": "Scooter",
  "latitude": 34.02161303565095,
  "longitude": -118.28327715396883,
  "phone": "2130000444",
  "status": "Lost"
}


In [21]:
search_filter = "date"
parameter = [2024, 4, 8, 2024, 4, 17]
search(search_filter, parameter)

Item ID: Lu7LqqhLMhU
{
  "color": "Black",
  "completed": false,
  "date": "2024-04-10",
  "description": "A Scooter",
  "email": "scooter@example.com",
  "image": "Placeholder...",
  "item_type": "Scooter",
  "latitude": 34.02161303565095,
  "longitude": -118.28327715396883,
  "phone": "2130000444",
  "status": "Lost"
}
Item ID: LXPoN6AmNdr
{
  "color": "Purple",
  "completed": false,
  "date": "2024-04-16",
  "description": "A Manager Add Test",
  "email": "Manager@example.com",
  "image": "Placeholder...",
  "item_type": "Scooter",
  "latitude": 0,
  "longitude": 0,
  "phone": "111111100000000",
  "status": "Lost"
}
Item ID: F7nyEse35le
{
  "color": "Black",
  "completed": false,
  "date": "2024-04-09",
  "description": "A black iphone 15",
  "email": "iphoneB@example.com",
  "image": "Placeholder...",
  "item_type": "Electronic Devices",
  "latitude": 34.0205148240064,
  "longitude": -118.28455388545991,
  "phone": "2130000111",
  "status": "Found"
}
Item ID: FkHUMHcX6oY
{
  "color

In [28]:
color = "Purple"
year = 2024
month = 4
day = 16
description = "A Manager Add Test"
email = "Manager@example.com"
item_type = "Scooter"
phone = "111111100000000"
status = "Lost"
date = datetime.datetime(year, month, day)
timestamp = date.timestamp()

item_json = {
  "color": color,
  "completed": False,
  "date": timestamp,
  "description": description,
  "email": email,
  "image": "",
  "item_type": item_type,
  "latitude": 0,
  "longitude": 0,
  "phone": phone,
  "status": status
}

# Add
add_edit(item_json=item_json)

# Edit
item_ID = "LXPoN6AmNdr"
add_edit(item_json=item_json, item_ID=item_ID)

Edit successfull!!


In [41]:
delete("L2kFe1tc9Vo")

---------Database 0------------
---------Database 1------------
---------Database 2------------


In [46]:
mark_as_resolved("LWPOsOepA2f")